# Newton

we want to create a symulation of 2 planets. To do so we have  a domain $\Omega$ and two bodies (ball-shaped) $B_1$ and $B_2$ with masses $m_1$ and $m_2$ respectively.

First we define some density $\rho$ on the planets


In [1]:
from ngsolve import *
from ngsolve import *
from netgen.csg import *
import scipy.sparse as sp
import matplotlib.pylab as plt
from zenith import * 


# import netgen.webgui  
# from ngsolve.webgui import Draw
import netgen.gui
%gui tk

importing NGSolve-6.2.2204
optfile ./ng.opt does not exist - using default values
togl-version : 2
OCC module loaded
loading ngsolve library
NGSolve-6.2.2204
Using Lapack
Including sparse direct solver Pardiso
Running parallel using 16 thread(s)


In [2]:
# create planet one and planet two
PL1 = Sphere((-2,0,0), 1).bc("PL_bc1").mat("planet1").maxh(0.3)
PL2 = Sphere(( 2,0,0), 1).bc("PL_bc2").mat("planet2").maxh(0.3)
Planets = PL1 + PL2
Planets.mat("Planets")
Void = Sphere((0,0,0), 10).bc("outer_bc") - PL1 - PL2
Void.mat("Void")

geo = CSGeometry()
geo.Add(Planets)
geo.Add(Void)

mesh = Mesh(geo.GenerateMesh(maxh=3, grading=0.4))
mesh.Curve(1)

clipping = {"z":-1}
Draw(mesh, clipping=clipping)

 Start Findpoints
 Analyze spec points
 Find edges
 Start Findpoints
 Analyze spec points
 Find edges
 Start Findpoints
 Analyze spec points
 Find edges
 Surface 1 / 3
 Optimize Surface
 Surface 2 / 3
 Optimize Surface
 Surface 3 / 3
 Optimize Surface
 Delaunay meshing
 Delaunay meshing
 start tetmeshing
 Success !
 Remove Illegal Elements
 Volume Optimization
 Curve elements, order = 1


In [3]:
RHO = L2(mesh, order=2, dirichlet="outer_bc")
U = H1(mesh, order=2, dirichlet="outer_bc")
V = VectorValued(U)

rho , drho = RHO.TnT()
u,  du = U.TnT()
v , dv = V.TnT()

# density of a planet
gfrho     = GridFunction(RHO)
gfrho.Set(CF((10)),definedon = mesh.Materials("Planets"),  bonus_intorder =10)
#Draw (gfrho, mesh, "rho", clipping=clipping)

# gravitational potential tp be solved
gfphi     = GridFunction(U)
gfphi.Set(CF((0)), BND ,  bonus_intorder =10)

# define the displacement field u
gfu = GridFunction(V)

# velocity field
gfv = GridFunction(V)
vel = CF((0,IfPos(x,1,-1),0))
gfv.Set(vel, definedon = mesh.Materials("Planets"),  bonus_intorder =10)
#Draw(gfv,mesh , clipping=clipping)


In [4]:



    


# A = sp.csr_matrix(mass_phi.mat.CSR())
# plt.rcParams['figure.figsize'] = (10,10)
# plt.spy(A)
# plt.show()


In [5]:
inverse = "sparsecholesky"

r = sqrt(x*x + y*y + z*z)
laplacian = BilinearForm(U)
laplacian += grad(u)*grad(du)*dx
laplacian.Assemble()
inv_laplacian = laplacian.mat.Inverse(U.FreeDofs(), inverse="sparsecholesky")

# mass rho0
mass_ss = BilinearForm(trialspace=RHO, testspace=U )
mass_ss += rho*du*dx
mass_ss.Assemble()

# gradient
gradient= BilinearForm(trialspace=V, testspace=U )
gradient += v*grad(du)*dx
gradient.Assemble()


# A = sp.csr_matrix(gradient.mat.CSR())
# kwargs = {'figsize': (10, 10), 'dpi': 300}
# Spy(A, n=20,**kwargs)




In [6]:
dt = 0.001
Draw(   gfv , mesh, "v")#, clipping=clipping)
with TaskManager():
    for i in range(100):
        gfphi.vec.data = inv_laplacian @ mass_ss.mat * gfrho.vec
        gfv.vec.data += dt*gradient.mat.T * gfphi.vec
        gfu.vec.data += dt*gfv.vec
        #scene.Redraw()
        Draw(gfv , mesh, "v")#, clipping=clipping)
        

#Draw(grad(gfphi) , mesh, clipping=clipping)


In [ ]:
# gravitational acc field



Draw(gfv,mesh, clipping=clipping)

In [ ]:
gfu.vec.data += dt*gfu.vec.data

In [ ]:
gfrho 